In [41]:
#imports

import pandas as pd
import numpy as np

First we will gather all the HSL bike data, and make it into a dataframe. 

In [42]:
# Read csv file into a pandas dataframe

bike_data = pd.DataFrame()
dataframes = []

for year in range(2016, 2022):
    if year > 2017:
        for month in range(4, 11):
            if month < 10:
                m = f'0{month}'
            else:
                m = str(month)
            file_name = './data/HSL/od-trips-' + str(year) + '/' + str(year) + '-' + m + '.csv'
            year_bike_data = pd.read_csv(file_name)
            dataframes.append(year_bike_data)
    else:
        for month in range(5, 11):
            if month < 10:
                m = f'0{month}'
            else:
                m = str(month)
            file_name = './data/HSL/od-trips-' + str(year) + '/' + str(year) + '-' + m + '.csv'
            year_bike_data = pd.read_csv(file_name)
            dataframes.append(year_bike_data)

bike_data = pd.concat(dataframes, ignore_index=True)
print(bike_data.head())
print(bike_data.shape)
            
        

/tmp/ipykernel_6658/3488630262.py:14: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  year_bike_data = pd.read_csv(file_name)


             Departure               Return Departure station id  \
0  2016-05-31T23:58:00  2016-06-01T00:13:00                  A27   
1  2016-05-31T23:58:00  2016-06-01T00:25:00                  A01   
2  2016-05-31T23:56:00  2016-06-01T00:08:00                  A40   
3  2016-05-31T23:55:00  2016-06-01T00:06:00                  B08   
4  2016-05-31T23:55:00  2016-06-01T00:07:00                  B08   

  Departure station name Return station id Return station name  \
0        Mannerheimintie               A35         Apollonkatu   
1            Kaivopuisto               A12         Unioninkatu   
2            Lastenlehto               A06           Viiskulma   
3  Sörnäisten metroasema               C02             Ooppera   
4  Sörnäisten metroasema               C02             Ooppera   

   Covered distance (m)  Duration (sec.)  
0                2585.0            944.0  
1                2703.0           1627.0  
2                1194.0            705.0  
3                2095.

Now we can get rid of some of the columns we don't need and drop any rows that have missing values.

In [43]:
#drop unnecessary columns
bike_data = bike_data.drop(['Return station id', 'Departure station id', 'Covered distance (m)', 'Duration (sec.)'], axis=1)

#drop rows with missing values
bike_data = bike_data.dropna()

Now we have our bike data in a dataframe. The next step is transformation, where we will make a new dataframe that contains the number of returns and departures per day per station.

In [44]:
#convert departure and return columns to datetime objects
bike_data['Departure'] = pd.to_datetime(bike_data['Departure'], format='mixed')
bike_data['Return'] = pd.to_datetime(bike_data['Return'], format='mixed')
bike_data['Departure'] = bike_data['Departure'].dt.date
bike_data['Return'] = bike_data['Return'].dt.date

#getting counts of departures and returns by station name and date
departure_counts = bike_data.groupby(['Departure station name', 'Departure']).size().reset_index(name='Departure Count')
return_counts = bike_data.groupby(['Return station name', 'Return']).size().reset_index(name='Return Count')

#combine departure and return counts by station name and date
combined_counts = pd.merge(departure_counts, return_counts, left_on=['Departure station name', 'Departure'], right_on=['Return station name', 'Return'], how='outer')

# Rename columns and fill NaN values with 0
combined_counts.rename(columns={'Departure station name': 'Station', 'Departure': 'Date'}, inplace=True)
combined_counts['Departure Count'].fillna(0, inplace=True)
combined_counts['Return Count'].fillna(0, inplace=True)

# Drop unnecessary columns
combined_counts.drop(['Return station name', 'Return'], axis=1, inplace=True)

#drop na values
combined_counts = combined_counts.dropna()

#drop any rows where Station columns contains 'station'
mask = combined_counts['Station'].str.contains('station', case=False)

# Use the mask to drop rows
combined_counts = combined_counts[~mask]

# Sort the combined DataFrame by date in ascending order
combined_counts.sort_values(by='Date', inplace=True)

# Reset the index
combined_counts.reset_index(drop=True, inplace=True)

# Display the resulting DataFrame
display(combined_counts)

# Save the DataFrame as a csv file
combined_counts.to_csv('./data/HSL/processed_bike_data.csv', index=False)
    


,Station,Date,Departure Count,Return Count
0,Linnanmäki,2016-05-02,11.0,15.0
1,Workshop Helsinki,2016-05-02,14.0,13.0
2,Kampin metroasema,2016-05-02,17.0,12.0
3,Laivasillankatu,2016-05-02,3.0,3.0
4,Merisotilaantori,2016-05-02,2.0,3.0
...,...,...,...,...
317556,Koivusaari (M),2021-10-31,7.0,10.0
317557,Komeetankatu,2021-10-31,8.0,8.0
317558,Toppelundintie,2021-10-31,1.0,4.0
317559,Torpanranta,2021-10-31,8.0,9.0
